In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#!pip install -U fvcore

In [4]:
#Importing the necessary libraries
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.gta5 import GTA5Custom
from datasets.cityscapes import CityscapesCustom
from models.bisenet.build_bisenet import BiSeNet
from models.discriminator import FCDiscriminator
from train_adversarial import train_adversarial
from utils import test_latency_FPS, test_FLOPs_params, plot_miou_over_epochs

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
gta5_height, gta5_width = (8, 16)
gta5_batch_size = 4

cityscapes_height, cityscapes_width = (8, 16)
cityscapes_batch_size = 4

n_epochs = 10

lambda_adv = 0.001

Using device: cpu


In [6]:
#Create Dataloaders for Cityscapes and GTA5
gta5_dir = os.path.dirname(os.getcwd()) + '/GTA5/GTA5/'
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

augment1 = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)
gta5_train_dataset_aug1 = GTA5Custom(gta5_dir, gta5_height, gta5_width, augment=augment1)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', cityscapes_height, cityscapes_width)

gta5_train_dataloader_aug1 = DataLoader(gta5_train_dataset_aug1, gta5_batch_size, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, cityscapes_batch_size, shuffle=False)

#Get the class names
class_names = cityscapes_test_dataset.get_class_names()

print(f'GTA5 (Train): {len(gta5_train_dataset_aug1)} images, divided into {len(gta5_train_dataloader_aug1)} batches of size {gta5_train_dataloader_aug1.batch_size}')
print(f'Cityscapes (Test): {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

GTA5 (Train): 2500 images, divided into 625 batches of size 4
Cityscapes (Test): 500 images, divided into 125 batches of size 4


In [7]:
#Set up the segmentation network (our Generator) with the pretrained weights
generator_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for the Generator
g_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
g_optimizer = torch.optim.SGD(generator_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [8]:
#Set up the Discriminators
discriminator_model = FCDiscriminator(num_classes=19).to(device)

#Set up the loss functions and the optimizers for the Discriminators
d_criterion = torch.nn.BCEWithLogitsLoss()
d_optimizer = torch.optim.Adam(discriminator_model.parameters(), lr=1e-4, betas=(0.9, 0.99))

In [ ]:
all_train_miou, all_test_miou, best_epoch = train_adversarial(generator_model, discriminator_model,
                                                              g_criterion, d_criterion, g_optimizer, d_optimizer, lambda_adv,
                                                              gta5_train_dataloader_aug1, cityscapes_test_dataloader,
                                                              class_names, device, n_epochs, model_name='Adversarial')